In [1]:
#Upgrade dependencies
!pip install --upgrade pip
!pip install --upgrade scikit-learn

     |████████████████████████████████| 1.5 MB 19.5 MB/s eta 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 21.0.1
    Uninstalling pip-21.0.1:
      Successfully uninstalled pip-21.0.1
     |████████████████████████████████| 22.2 MB 17.4 MB/s eta 0:00:01
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.24.1
    Uninstalling scikit-learn-0.24.1:
      Successfully uninstalled scikit-learn-0.24.1


In [2]:
! pip install boto3

In [3]:
import boto3

client = boto3.client('comprehend')

In [6]:
response = client.detect_sentiment(
    Text='string',
    LanguageCode='en'
)

response

{'Sentiment': 'NEUTRAL',
 'SentimentScore': {'Positive': 0.011055116541683674,
  'Negative': 0.013187303207814693,
  'Neutral': 0.9740014672279358,
  'Mixed': 0.001756161917001009},
 'ResponseMetadata': {'RequestId': '5a3515bd-bf71-45fc-850e-c5e4d6beca45',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '5a3515bd-bf71-45fc-850e-c5e4d6beca45',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '164',
   'date': 'Thu, 10 Jun 2021 09:07:07 GMT'},
  'RetryAttempts': 0}}

In [7]:
response['Sentiment']

'NEUTRAL'

In [9]:
import numpy as np                                
import matplotlib.pyplot as plt   
from IPython.display import Image                 
from IPython.display import display               
from sklearn.datasets import dump_svmlight_file   
from time import gmtime, strftime                 
import sys                                        
import math                                       
import json
import boto3
import sagemaker

bucket = sagemaker.Session().default_bucket()
prefix = 'SageMaker/'

print('Bucket:\n{}'.format(bucket))


Bucket:
sagemaker-eu-west-1-461627159302


In [13]:
! ls

1429_1.csv		   Sample NB_Text Classifiers.ipynb  Untitled.ipynb
NLP-TEXT-PROCESS-NB.ipynb  TF-IDF.ipynb


In [14]:
import pandas as pd

import warnings
warnings.filterwarnings("ignore")
  
df = pd.read_csv('1429_1.csv')

print('The shape of the training dataset is:', df.shape)

The shape of the training dataset is: (34660, 21)


In [16]:
# Sample observations
df.head(3)

,id,name,asins,brand,categories,keys,manufacturer,reviews.date,reviews.dateAdded,reviews.dateSeen,...,reviews.doRecommend,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.userCity,reviews.userProvince,reviews.username
0,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-13T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",...,True,NaN,0.0,5.0,http://reviews.bestbuy.com/3545/5620406/review...,This product so far has not disappointed. My c...,Kindle,NaN,NaN,Adapter
1,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-13T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",...,True,NaN,0.0,5.0,http://reviews.bestbuy.com/3545/5620406/review...,great for beginner or experienced person. Boug...,very fast,NaN,NaN,truman
2,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-13T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",...,True,NaN,0.0,5.0,http://reviews.bestbuy.com/3545/5620406/review...,Inexpensive tablet for him to use and learn on...,Beginner tablet for our 9 year old son.,NaN,NaN,DaveZ


In [25]:
# df.loc[:, df.columns == 'reviews.text' or  'reviews.rating' ]
df = df[['reviews.text', 'reviews.rating']]

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34660 entries, 0 to 34659
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   reviews.text    34659 non-null  object 
 1   reviews.rating  34627 non-null  float64
dtypes: float64(1), object(1)
memory usage: 541.7+ KB


In [27]:
df.isnull().sum()

reviews.text       1
reviews.rating    33
dtype: int64

In [28]:
df.dropna(axis=0, inplace= True)

In [29]:
df.isnull().sum()

reviews.text      0
reviews.rating    0
dtype: int64

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34626 entries, 0 to 34659
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   reviews.text    34626 non-null  object 
 1   reviews.rating  34626 non-null  float64
dtypes: float64(1), object(1)
memory usage: 811.5+ KB


In [31]:
df['reviews.text']

0        This product so far has not disappointed. My c...
1        great for beginner or experienced person. Boug...
2        Inexpensive tablet for him to use and learn on...
3        I've had my Fire HD 8 two weeks now and I love...
4        I bought this for my grand daughter when she c...
                               ...                        
34655    This is not appreciably faster than any other ...
34656    Amazon should include this charger with the Ki...
34657    Love my Kindle Fire but I am really disappoint...
34658    I was surprised to find it did not come with a...
34659    to spite the fact that i have nothing but good...
Name: reviews.text, Length: 34626, dtype: object

In [48]:
df_2000 = df['reviews.text'].sample(n=2000)

In [50]:
rev_list = df_2000.values.tolist()

rev_list[0:25]

['I love everything about my fire tv. I wish I wouldve gotten one sooner!',
 'My 5 year old sons love it , Seems to be well made .',
 "I've bought 2 of these tablets now for my kids, they're a great value for its low price. Great screen quality, everything you need in a tablet and good size. Very happy with them.",
 'Bought one for each of my kids and they love it. Functionally sound product very easy to use.',
 "I picked this up on Black Friday when it was $30. It seemed like the parental controls would make it good for my kids. I'm not sold. While I can sideload the Google Play store, you can't access anything you download from the safety of the child's account. In addition, 8 GB is practically useless.",
 'i mean if you want just a tablet to read some books and browse internet thats fine.',
 'For the price you pay and to do basic functions this table is well worth the money.',
 'I love it. Use it daily for everything. Love the options and looks.',
 "I bought this tablet for my daugh

In [52]:
output = []
for txt in rev_list:
    
    response2 = client.detect_sentiment(
                Text=txt,
                LanguageCode='en'
                ) 
    output.append(response2['Sentiment'])

output

['POSITIVE',
 'POSITIVE',
 'POSITIVE',
 'POSITIVE',
 'NEGATIVE',
 'NEUTRAL',
 'POSITIVE',
 'POSITIVE',
 'POSITIVE',
 'POSITIVE',
 'POSITIVE',
 'NEUTRAL',
 'POSITIVE',
 'POSITIVE',
 'POSITIVE',
 'NEGATIVE',
 'POSITIVE',
 'POSITIVE',
 'POSITIVE',
 'POSITIVE',
 'POSITIVE',
 'MIXED',
 'POSITIVE',
 'MIXED',
 'MIXED',
 'POSITIVE',
 'NEGATIVE',
 'POSITIVE',
 'POSITIVE',
 'POSITIVE',
 'POSITIVE',
 'MIXED',
 'POSITIVE',
 'POSITIVE',
 'POSITIVE',
 'MIXED',
 'MIXED',
 'POSITIVE',
 'POSITIVE',
 'POSITIVE',
 'POSITIVE',
 'MIXED',
 'POSITIVE',
 'POSITIVE',
 'POSITIVE',
 'POSITIVE',
 'POSITIVE',
 'MIXED',
 'POSITIVE',
 'POSITIVE',
 'POSITIVE',
 'NEGATIVE',
 'POSITIVE',
 'POSITIVE',
 'POSITIVE',
 'MIXED',
 'POSITIVE',
 'POSITIVE',
 'POSITIVE',
 'POSITIVE',
 'POSITIVE',
 'MIXED',
 'POSITIVE',
 'POSITIVE',
 'POSITIVE',
 'POSITIVE',
 'POSITIVE',
 'MIXED',
 'POSITIVE',
 'POSITIVE',
 'MIXED',
 'POSITIVE',
 'POSITIVE',
 'POSITIVE',
 'POSITIVE',
 'POSITIVE',
 'POSITIVE',
 'POSITIVE',
 'POSITIVE',
 'POSITIVE'

In [53]:
df_output = pd.DataFrame()
df_output['text'] = rev_list
df_output['sentiment'] = output

In [54]:
df_output.head(10)

,text,sentiment
0,I love everything about my fire tv. I wish I w...,POSITIVE
1,"My 5 year old sons love it , Seems to be well ...",POSITIVE
2,I've bought 2 of these tablets now for my kids...,POSITIVE
3,Bought one for each of my kids and they love i...,POSITIVE
4,I picked this up on Black Friday when it was $...,NEGATIVE
5,i mean if you want just a tablet to read some ...,NEUTRAL
6,For the price you pay and to do basic function...,POSITIVE
7,I love it. Use it daily for everything. Love t...,POSITIVE
8,I bought this tablet for my daughter to watch ...,POSITIVE
9,My nephew always uses my sister's iphone when ...,POSITIVE


In [57]:
df_output.to_csv("output.csv")